regex는 머지


In [35]:
import os
import math
import random
import pandas as pd
import regex as re
from typing import Optional, Sequence

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support,\
                             roc_auc_score,f1_score

from tqdm import tqdm
import torch
import torch.nn as nn
from torch import Tensor
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import DataLoader, Dataset, random_split
from torch.nn import functional as F

from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification,\
                        AutoModelForSequenceClassification,AutoTokenizer, EarlyStoppingCallback,\
                        AutoModel,AutoConfig,T5TokenizerFast,ElectraTokenizerFast,ElectraModel,\
                        T5ForConditionalGeneration

import gc
os.environ["TOKENIZERS_PARALLEISM"] = "false"              

In [22]:
data_dir = "./dataset/"

train = pd.read_csv(os.path.join(data_dir,"train.csv")).drop(['ID'],axis=1)
test = pd.read_csv(os.path.join(data_dir, "test.csv"))

In [28]:
train

,문장,유형,극성,시제,확실성,label
0,누군가는 누군가는 살아남고 무대를 떠났다,사실형,긍정,과거,확실,사실형-긍정-과거-확실
1,현재 약 200여 명의 한국생애설계사들이 교육컨설턴트 상담사 등으로 활동하고 있으며...,사실형,긍정,현재,확실,사실형-긍정-현재-확실
2,한 활성화돼 말했다 업계의 어려움을 이해해주어 공항 면세점 임대료 감면을 연장해 주...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
3,캐릭터 나열 순서 설정 기능을 통해 각 이용자가 보유한 캐릭터 중 원하는 캐릭터를 ...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
4,정도로 관람객이 거의 없을 주말이지만 한적했다,사실형,긍정,과거,확실,사실형-긍정-과거-확실
...,...,...,...,...,...,...
64692,생명을 다투는 위기에 정쟁은 무의미하다,추론형,긍정,현재,확실,추론형-긍정-현재-확실
64693,조이시티는 코로나19 관련 상황이 안정화될 부서별로 각 때까지 탄력적으로 재택근무를...,사실형,긍정,현재,확실,사실형-긍정-현재-확실
64694,이 가운데 심사를 거쳐 사회적기업으로 인증된 19곳이다 곳은,사실형,긍정,현재,확실,사실형-긍정-현재-확실
64695,코로나19 사태가 언제까지 오리무중이다 지속될지,추론형,긍정,현재,확실,추론형-긍정-현재-확실


In [13]:
train.유형.unique(), train.극성.unique(), train.시제.unique(), train.확실성.unique()

(array(['사실형', '추론형', '예측형', '대화형'], dtype=object),
 array(['긍정', '부정', '미정'], dtype=object),
 array(['현재', '과거', '미래'], dtype=object),
 array(['확실', '불확실'], dtype=object))

In [23]:
train['문장'] = train['문장'].apply(lambda x: re.sub("[^ A-Za-z0-9가-힣]","",x))
train['문장'] = train['문장'].apply(lambda x: re.sub("[ +]"," ",x))

test['문장'] = test['문장'].apply(lambda x: re.sub("[^ A-Za-z0-9가-힣]","",x))
test['문장'] = test['문장'].apply(lambda x: re.sub("[ +]"," ",x))

In [24]:
train['문장']

0                       075포인트 금리 인상은 1994년 이후 28년 만에 처음이다
1        이어 앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정이라며 그 이전이라도 ...
2        정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30에서 37까지 확대한다
3        서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만 하루 만에 차...
4                  익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다
                               ...                        
16536    신동덤은 신비한 동물사전과 해리 포터 시리즈를 잇는 마법 어드벤처물로 전편에 이어 ...
16537    수족냉증은 어릴 때부터 심했으며 관절은 어디 한 곳이 아니고 목 어깨 팔꿈치 등 허...
16538    김금희 소설가는 계약서 조정이 그리 어려운가 작가를 격려한다면서 그런 문구 하나 고...
16539    1만명이 넘는 방문자수를 기록한 이번 전시회는 총 77개 작품을 넥슨 사옥을 그대로...
16540                                           목민심서의 내용이다
Name: 문장, Length: 16541, dtype: object

In [25]:
# https://github.com/catSirup/KorEDA/blob/master/eda.py
def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0

    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            return new_words

    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
    return new_words

def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)
    return new_words

def text_aug(sentence, alpha_rs = 0.1, num_aug=3):
    words = sentence.split(' ')
    words = [word for word in words if word != ""]
    num_words = len(words)

    augmented_sentences = []
    num_new_per_technique = num_aug

    n_rs = max(1, int(alpha_rs*num_words))

    for _ in range(num_new_per_technique):
        a_words = random_swap(words, n_rs)
        augmented_sentences.append(" ".join(a_words))

    augmented_sentences = [sentence for sentence in augmented_sentences]
    random.shuffle(augmented_sentences)

    if num_aug >= 1:
        augmented_sentences = augmented_sentences[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentences)
        augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
    return augmented_sentences

aug = train['문장'].apply(lambda x: text_aug(x))

In [26]:
tmp1 = train.copy()
tmp1['문장'] = list(map(lambda x: x[0], aug))

tmp2 = train.copy()
tmp2['문장'] = list(map(lambda x: x[1], aug))

tmp3 = train.copy()
tmp3['문장'] = list(map(lambda x: x[2], aug))

In [27]:
train = pd.concat([train,tmp1,tmp2,tmp3]).drop_duplicates(keep='first').sample(frac=1).reset_index(drop=True)
train

,문장,유형,극성,시제,확실성,label
0,누군가는 누군가는 살아남고 무대를 떠났다,사실형,긍정,과거,확실,사실형-긍정-과거-확실
1,현재 약 200여 명의 한국생애설계사들이 교육컨설턴트 상담사 등으로 활동하고 있으며...,사실형,긍정,현재,확실,사실형-긍정-현재-확실
2,한 활성화돼 말했다 업계의 어려움을 이해해주어 공항 면세점 임대료 감면을 연장해 주...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
3,캐릭터 나열 순서 설정 기능을 통해 각 이용자가 보유한 캐릭터 중 원하는 캐릭터를 ...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
4,정도로 관람객이 거의 없을 주말이지만 한적했다,사실형,긍정,과거,확실,사실형-긍정-과거-확실
...,...,...,...,...,...,...
64692,생명을 다투는 위기에 정쟁은 무의미하다,추론형,긍정,현재,확실,추론형-긍정-현재-확실
64693,조이시티는 코로나19 관련 상황이 안정화될 부서별로 각 때까지 탄력적으로 재택근무를...,사실형,긍정,현재,확실,사실형-긍정-현재-확실
64694,이 가운데 심사를 거쳐 사회적기업으로 인증된 19곳이다 곳은,사실형,긍정,현재,확실,사실형-긍정-현재-확실
64695,코로나19 사태가 언제까지 오리무중이다 지속될지,추론형,긍정,현재,확실,추론형-긍정-현재-확실


In [41]:
device = torch.device("cuda")
checkpoint = "kykim/electra-kor-base"
if checkpoint == 'paust/pko-t5-large':
    tokenozer = T5TokenizerFast.from_pretrained(checkpoint)
elif checkpoint == 'kykim/electra_kor_base':
    tokenizer = ElectraTokenizerFast.from_pretrained(checkpoint)
else:
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)

length = train['문장'].str.len().max()

In [55]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self,idx):
        item = {key:torch.tensor(val[idx]) for key,val in self.encodings.items()}
        if self.labels:
            st_type = self.labels['type'][idx]
            st_polarity = self.labels['polarity'][idx]
            st_tense = self.labels["tense"][idx]
            st_certainty = self.labels["certainty"][idx]
            item["labels"] = torch.tensor(st_type), torch.tensor(st_polarity), torch.tensor(st_tense), torch.tensor(st_certainty)
        
        return item
    def __len__(self):
        return len(self.encodings["input_ids"])

In [53]:
class FocalLoss(nn.Module):
    """ Focal Loss, as described in https://arxiv.org/abs/1708.02002.
    It is essentially an enhancement to cross entropy loss and is
    useful for classification tasks when there is a large class imbalance.
    x is expected to contain raw, unnormalized scores for each class.
    y is expected to contain class labels.
    Shape:
        - x: (batch_size, C) or (batch_size, C, d1, d2, ..., dK), K > 0.
        - y: (batch_size,) or (batch_size, d1, d2, ..., dK), K > 0.
    """

    def __init__(self,
                 alpha: Optional[Tensor] = None,
                 gamma: float = 0.,
                 reduction: str = 'mean',
                 ignore_index: int = -100):
        """Constructor.
        Args:
            alpha (Tensor, optional): Weights for each class. Defaults to None.
            gamma (float, optional): A constant, as described in the paper.
                Defaults to 0.
            reduction (str, optional): 'mean', 'sum' or 'none'.
                Defaults to 'mean'.
            ignore_index (int, optional): class label to ignore.
                Defaults to -100.
        """
        if reduction not in ('mean', 'sum', 'none'):
            raise ValueError(
                'Reduction must be one of: "mean", "sum", "none".')

        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ignore_index = ignore_index
        self.reduction = reduction

        self.nll_loss = nn.NLLLoss(
            weight=alpha, reduction='none', ignore_index=ignore_index)

    def __repr__(self):
        arg_keys = ['alpha', 'gamma', 'ignore_index', 'reduction']
        arg_vals = [self.__dict__[k] for k in arg_keys]
        arg_strs = [f'{k}={v!r}' for k, v in zip(arg_keys, arg_vals)]
        arg_str = ', '.join(arg_strs)
        return f'{type(self).__name__}({arg_str})'

    def forward(self, x: Tensor, y: Tensor) -> Tensor:
        if x.ndim > 2:
            # (N, C, d1, d2, ..., dK) --> (N * d1 * ... * dK, C)
            c = x.shape[1]
            x = x.permute(0, *range(2, x.ndim), 1).reshape(-1, c)
            # (N, d1, d2, ..., dK) --> (N * d1 * ... * dK,)
            y = y.view(-1)

        unignored_mask = y != self.ignore_index
        y = y[unignored_mask]
        if len(y) == 0:
            return torch.tensor(0.)
        x = x[unignored_mask]

        # compute weighted cross entropy term: -alpha * log(pt)
        # (alpha is already part of self.nll_loss)
        log_p = F.log_softmax(x, dim=-1)
        ce = self.nll_loss(log_p, y)

        # get true class column from each row
        all_rows = torch.arange(len(x))
        log_pt = log_p[all_rows, y]

        # compute focal term: (1 - pt)^gamma
        pt = log_pt.exp()
        focal_term = (1 - pt)**self.gamma

        # the full loss: -alpha * ((1 - pt)^gamma) * log(pt)
        loss = focal_term * ce

        if self.reduction == 'mean':
            loss = loss.mean()
        elif self.reduction == 'sum':
            loss = loss.sum()

        return loss

def focal_loss(alpha: Optional[Sequence] = None,
               gamma: float = 0.,
               reduction: str = 'mean',
               ignore_index: int = -100,
               device='cpu',
               dtype=torch.float32) -> FocalLoss:
    """Factory function for FocalLoss.
    Args:
        alpha (Sequence, optional): Weights for each class. Will be converted
            to a Tensor if not None. Defaults to None.
        gamma (float, optional): A constant, as described in the paper.
            Defaults to 0.
        reduction (str, optional): 'mean', 'sum' or 'none'.
            Defaults to 'mean'.
        ignore_index (int, optional): class label to ignore.
            Defaults to -100.
        device (str, optional): Device to move alpha to. Defaults to 'cpu'.
        dtype (torch.dtype, optional): dtype to cast alpha to.
            Defaults to torch.float32.
    Returns:
        A FocalLoss object
    """
    if alpha is not None:
        if not isinstance(alpha, Tensor):
            alpha = torch.tensor(alpha)
        alpha = alpha.to(device=device, dtype=dtype)

    fl = FocalLoss(
        alpha=alpha,
        gamma=gamma,
        reduction=reduction,
        ignore_index=ignore_index)
    return fl
        
def compute_metrics(pred):
    # label = [[cls1,cls2,...],]
    # preds = n list
    focal_loss = FocalLoss()
    labels = pred.label_ids
    preds = pred.predictions
    f1 = []
    focal = []
    for i in range(4):
        # focal.append(focal_loss(torch.tensor(preds[i], dtype=torch.float), torch.tensor(labels[::, i],dtype=torch.float)))
        f1.append(f1_score(y_true = labels[::, i], y_pred = preds[i], average='weighted'))
    return {
        #'focal': sum(focal),
        'f1-sum': sum(f1)/4
    }


In [34]:
config = AutoConfig.from_pretrained(checkpoint)
config.name_or_path="kr.kim"
print(config)

ElectraConfig {
  "_name_or_path": "kr.kim",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 42000
}



In [48]:
유형 = LabelEncoder()
유형.fit(train['유형'])

극성 = LabelEncoder()
극성.fit(train['극성'])

시제 = LabelEncoder()
시제.fit(train['시제'])

확실성 = LabelEncoder()
확실성.fit(train['확실성'])

def encoding(X_train, X_val):
    X_train["유형"] = 유형.transform(X_train['유형'])
    X_val["유형"] = 유형.transform(X_val["유형"])

    X_train["극성"] = 극성.transform(X_train['극성'])
    X_val["극성"] = 극성.transform(X_val["극성"])

    X_train["시제"] = 시제.transform(X_train['시제'])
    X_val["시제"] = 시제.transform(X_val["시제"])

    X_train["확실성"] = 확실성.transform(X_train['확실성'])
    X_val["확실성"] = 확실성.transform(X_val["확실성"])

    train_labels = {
        'type': X_train['유형'].values,
        'polarity': X_train['극성'].values,
        'tense': X_train['시제'].values,
        'certainty': X_train['확실성'].values
    }

    val_labels = {
        'type': X_val['유형'].values,
        'polarity': X_val['극성'].values,
        'tense': X_val['시제'].values,
        'certainty': X_val['확실성'].values
    }

    return train_labels, val_labels

In [21]:
def recent_file(path):
    file_name_and_time_lst = []
    # 해당 경로에 있는 파일들을 생성시간을 함꼐 리스트로 넣어주고 역순으로 정렬
    for f_name in os.listdir(f"{path}"):
        written_time = os.path.getctime(f"{path}/{f_name}")
        file_name_and_time_lst.append((f_name, written_time))

    sorted_file_lst = sorted(file_name_and_time_lst, key=lambda x:x[1], reverse=True)
    recent_file = sorted_file_lst[0]
    recent_file_name = recent_file[0]
    return f"{path}/{recent_file_name}"

In [22]:
test_results = []

In [57]:
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        if checkpoint == 'monologg/kobigbird-bert-case':
            config.attention_type = "original_full"
            self.base_model = AutoModel.from_pretrained(checkpoint, config=config)
        elif checkpoint == 'kykim/electra-kor-base':
            self.base_model = ElectraModel.from_pretrained(checkpoint, config=config)
        elif checkpoint == 'paust/pko-t5-large':
            self.base_model = T5ForConditionalGeneration.from_pretrained(checkpoint, config=config)
        else:
            self.base_model = AutoModel.from_pretrained(checkpoint, config=config)
       
        try:
            self.out = self.base_model.encoder.layer[-1].output.dense.out_features
        except:
            self.out = 768
        
        self.type_classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=self.out, out_features=4)
        )
        self.polarity_classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=self.out, out_features=3)
        )
        self.tense_classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=self.out, out_features=3)
        )
        self.certainty_classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=self.out, out_features=2)
        )

    def forward(self, input_ids, attention_mask, labels=None, token_type_ids=None):
        if checkpoint == "paust/pko-t5-large":
            x = self.base_model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=input_ids)[0]
        else:
            x = self.base_model(input_ids=input_ids, attention_mask=attention_mask)[0]

        type_output = self.type_classifier(x[:,0,:].view(-1,self.out))
        polarity_output = self.polarity_classifier(x[:,0,:].view(-1,self.out))
        tense_output = self.tense_classifier(x[:,0,:].view(-1,self.out))
        certainty_output = self.certainty_classifier(x[:,0,:].view(-1,self.out))
        return type_output, polarity_output, tense_output, certainty_output


In [61]:
lr = 1e-4
stop = 3
epoch = 1000
batch = 16
seed = 42

In [24]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
test_tokenized = tokenizer(test["문장"].tolist(),padding=True, truncation=True, max_length=length, return_tensors="pt")
test_dataset = Cust

In [59]:

class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args,**kwargs)

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels").to(torch.int64)
        type_logit, polarity_logit, tense_logit, certainty_logit = model(**inputs)

        criterion = {
            'type' : FocalLoss().to(device),
            'polarity': FocalLoss().to(device),
            'tense': FocalLoss().to(device),
            'certainty': FocalLoss().to(device)
        }
        loss = criterion['type'](type_logit, labels[::,0]) + \
                criterion['polarity'](polarity_logit, labels[::,1]) + \
                criterion['tense'](tense_logit,labels[::,2]) + \
                criterion['certainty'](certainty_logit,labels[::,3])
        
        outputs = None,\
                torch.argmax(type_logit, dim=1),\
                torch.argmax(polarity_logit, dim=1),\
                torch.argmax(tense_logit, dim=1),\
                torch.argmax(certainty_logit,dim=1)
        
        return (loss, outputs) if return_outputs else loss

In [62]:
print(f'hidden_layers : {config.num_hidden_layers}')
# config.num_hidden_layers = 10
print(f'now_hidden_layers : {config.num_hidden_layers}')

kf = KFold(n_splits=5, random_state=seed, shuffle=True)
for i, (train_index, test_index) in enumerate(kf.split(train)):
    print(f'Round {i}')
    X_train, X_val = train.loc[train_index, :], train.loc[test_index, :]
    train_labels, val_labels = encoding(X_train, X_val)
    token_train, token_val = tokenizer(X_train.문장.tolist(), padding=True, truncation=True, max_length=length), tokenizer(X_val.문장.tolist(), padding=True, truncation=True, max_length=length)
    train_dataset, val_dataset = CustomDataset(token_train, train_labels), CustomDataset(token_val, val_labels)
    model = CustomModel()
    model.to(device)
    args = TrainingArguments(run_name = f'fold_{i}',                                # 모델이름
                             output_dir= f"fold_{i}",                               # 모델저장경로
                             evaluation_strategy="steps",                           # 모델의 평가를 언제 진행할지
                             eval_steps=100,                                        # 500 스텝 마다 모델 평가
                             save_steps=100,                                        # 500 스텝 마다 모델 저장
                             save_total_limit = 2,                                  # 저장할 모델의 갯수
                             logging_steps=100,                                     # 학습로스 로깅
                             per_device_train_batch_size=batch,                     # GPU에 학습데이터를 몇개씩 올려서 학습할지
                             per_device_eval_batch_size=batch,                      # GPU에 학습데이터를 몇개씩 올려서 평가할지
                             gradient_accumulation_steps=16,                        # 가상배치
                             num_train_epochs=epoch,                                # 전체 학습 진행 횟수
                             learning_rate=lr,                                      # 학습률 정의 
                             seed=seed,                                             # seed
                             load_best_model_at_end=True,                           # 평가기준 스코어가 좋은 모델만 저장할지 여부
                             fp16=True,
                             do_train=True,
                             do_eval=True,
                             # metric_for_best_model
                             # greater_is_better = True,
    )
    trainer = CustomTrainer(model=model,
                            args=args,                                                        # args
                            train_dataset=train_dataset,                                      # 학습데이터
                            eval_dataset=val_dataset,                                         # validation 데이터
                            compute_metrics=compute_metrics,                                  # 모델 평가 방식
                            callbacks=[EarlyStoppingCallback(early_stopping_patience=stop)],) # callback
    trainer.train()
    del model
    del trainer
    gc.collect() # python 자원 관리 
    torch.cuda.empty_cache() # gpu 자원관리   

hidden_layers : 12
now_hidden_layers : 12
Round 0


loading weights file pytorch_model.bin from cache at C:\Users\kyj09/.cache\huggingface\hub\models--kykim--electra-kor-base\snapshots\8599418d72f5dcb21ae3972ba2405f88c819b195\pytorch_model.bin
Some weights of the model checkpoint at kykim/electra-kor-base were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of ElectraModel were initialized from

  0%|          | 0/202000 [00:00<?, ?it/s]

***** Running Evaluation *****
  Num examples = 12940
  Batch size = 16


{'loss': 1.3559, 'learning_rate': 9.995148514851486e-05, 'epoch': 0.49}


...

{'loss': 0.5496, 'learning_rate': 9.985247524752476e-05, 'epoch': 1.48}


  0%|          | 0/809 [00:00<?, ?it/s]

Saving model checkpoint to fold_0\checkpoint-300
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


{'eval_loss': 0.5150865912437439, 'eval_f1-sum': 0.956678700559104, 'eval_runtime': 54.9087, 'eval_samples_per_second': 235.664, 'eval_steps_per_second': 14.734, 'epoch': 1.48}


Deleting older checkpoint [fold_0\checkpoint-100] due to args.save_total_limit


In [ ]:
def recent_file(path):
    file_name_and_time_lst = []
    # 해당 경로에 있는 파일들의 생성시간을 함께 리스트로 넣어줌. 
    for f_name in os.listdir(f"{path}"):
        written_time = os.path.getctime(f"{path}/{f_name}")
        file_name_and_time_lst.append((f_name, written_time))
    # 생성시간 역순으로 정렬하고, 
    sorted_file_lst = sorted(file_name_and_time_lst, key=lambda x: x[1], reverse=True)
    # 가장 앞에 이는 놈을 넣어준다.
    recent_file = sorted_file_lst[0]
    recent_file_name = recent_file[0]
    return f"{path}/{recent_file_name}"

In [ ]:
gc.collect() # python 자원 관리 
torch.cuda.empty_cache() # gpu 자원관리
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenized = tokenizer(test.문장.tolist(), padding=True, truncation=True, max_length=length, return_tensors="pt")
test_dataset = CustomDataset(tokenized, None)
test_args = TrainingArguments(
    output_dir = './',
    do_train = False,
    do_predict = True,
    per_device_eval_batch_size = 512,   
    dataloader_drop_last = False    
)

tmp = 0
while os.path.isdir(f'fold_{tmp}'):
    tmp += 1

test_results = []
for i in range(tmp):
    print(f'Round {i}')
    # model = AutoModel.from_pretrained(recent_file('custom_model'), config=config)
    model = CustomModel().to(device)
    model.load_state_dict(torch.load(f"{recent_file(f'fold_{i}')}/pytorch_model.bin"))
    trainer = CustomTrainer(
                  model = model, 
                  args = test_args, 
                  compute_metrics = compute_metrics)
    test_results.append(trainer.predict(test_dataset))
    del model
    del trainer
    gc.collect() # python 자원 관리 
    torch.cuda.empty_cache() # gpu 자원관리

In [ ]:
test['유형'] = list(map(lambda x : 유형.inverse_transform([np.argmax(x)]), sum(list(map(lambda x: x.predictions[0], test_results)))/len(test_results)))
test['극성'] = list(map(lambda x : 극성.inverse_transform([np.argmax(x)]), sum(list(map(lambda x: x.predictions[1], test_results)))/len(test_results)))
test['시제'] = list(map(lambda x : 시제.inverse_transform([np.argmax(x)]), sum(list(map(lambda x: x.predictions[2], test_results)))/len(test_results)))
test['확실성'] = list(map(lambda x : 확실성.inverse_transform([np.argmax(x)]), sum(list(map(lambda x: x.predictions[3], test_results)))/len(test_results)))

test['유형'] = list(map(lambda x : x[0], test['유형']))
test['극성'] = list(map(lambda x : x[0], test['극성']))
test['시제'] = list(map(lambda x : x[0], test['시제']))
test['확실성'] = list(map(lambda x : x[0], test['확실성']))

In [ ]:
test['label'] = test['유형'] + '-' + test['극성'] + '-' + test['시제'] + '-' + test['확실성']
test

In [ ]:
sub = pd.read_csv('sample_submission.csv')
sub['label'] = test['label']
tmp = 0
while os.path.exists(f'제출{tmp}.csv'):
    tmp += 1
sub.to_csv(f'제출{tmp}.csv', index=False, mode='w')